互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

    contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
    is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
    is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
    contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
    contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
    contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
    contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
    is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
    with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
    contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
    contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
    contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
    allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
    contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
    contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
    has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
    traffic：该网站的流量大小。1表示大，0表示小；
    google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；
    
    y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
    原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [180]:
import pandas as pd

# 读取fraudulent.csv
df = pd.read_csv('data/fraudulent.csv')

# 查看各列的缺失值比例
missing_ratio = df.isnull().mean()
print(missing_ratio)

contain_IP               0.008923
is_long                  0.008824
is_tinyurl               0.008725
contain_at               0.008130
contain_double_slash     0.011501
contain_dash             0.009320
contain_subdomain        0.009617
is_SSL                   0.009518
with_long_history        0.277117
contain_icon             0.134642
contain_ext_domain       0.151398
contain_email_to         0.206127
allow_right_click        0.337795
contain_pop_up_window    0.027662
contain_Iframe           0.065338
has_DNSRecord            0.119076
traffic                  0.149415
google_rank              0.065834
y                        0.000000
dtype: float64


In [181]:
# 删除缺失值比例大于0.2的列，用众数填补其他缺失值
df = df.loc[:, missing_ratio <= 0.2]
for col in df.columns:
    mode = df[col].mode()[0]    # 取众数：df[col].mode是一个Series对象
    df[col] = df[col].fillna(mode)

# 再次查看缺失值情况
print(df.isnull().mean())

contain_IP               0.0
is_long                  0.0
is_tinyurl               0.0
contain_at               0.0
contain_double_slash     0.0
contain_dash             0.0
contain_subdomain        0.0
is_SSL                   0.0
contain_icon             0.0
contain_ext_domain       0.0
contain_pop_up_window    0.0
contain_Iframe           0.0
has_DNSRecord            0.0
traffic                  0.0
google_rank              0.0
y                        0.0
dtype: float64


In [182]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# 分离特征和标签
X = df.drop('y', axis=1)
y = df['y']
# 将原始数据分为训练集、测试集（随机种子请设置为1）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 各种模型
models = {
    'K近邻  ': KNeighborsClassifier(),
    '决策树  ': DecisionTreeClassifier(),
    '逻辑回归': LogisticRegression(),
    '支持向量机': SVC(),
    '随机森林': RandomForestClassifier(),
    '梯度提升': GradientBoostingClassifier(),
    '多层感知器': MLPClassifier(max_iter=500),
    '极限梯度提升': XGBClassifier()
}

print("模型名称  \t\t F1得分")
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    print(f'{model_name} \t\t {f1}')

模型名称  		 F1得分
K近邻   		 0.8412822517591869
决策树   		 0.8715083798882681
逻辑回归 		 0.85
支持向量机 		 0.8610223642172524
随机森林 		 0.8719172633253779
梯度提升 		 0.861198738170347
多层感知器 		 0.8691514670896114
极限梯度提升 		 0.8723235527359239
